#### We use the Dataset provided by the British Health Authoritites to find information about the Geolocation of each Tesco Store in the London Area. 
Other Ways seemed rather hard to get the Location data

In [21]:
import pandas as pd
import xml.etree.ElementTree as et
import urllib.request
import pandas_read_xml as pdrx
import numpy as np

In [53]:
## Iterate over the LocalAuthorityCodes for the Greater London Area (501 - 533)
df_tesco_store_locations = pd.DataFrame()

for x in range(501, 534):
    
    url =  "https://ratings.food.gov.uk/OpenDataFiles/FHRS{0}en-GB.xml".format(x)
    print("Loading Data for Municipality ", x, "From URL: ", url)
    
    # Load the DataSet directly from the XML File
    df = pdrx.read_xml(url, ['FHRSEstablishment', 'EstablishmentCollection'])
    
    # Parsing the XML
    df["BusinessName"] = df["EstablishmentDetail"].apply(lambda x : x["BusinessName"])
    df["GeoCode"] = df["EstablishmentDetail"].apply(lambda x : x["Geocode"])
    df["Longitude"] = df["GeoCode"].apply(lambda x : x["Longitude"] if x is not None else np.nan)
    df["Latitude"] = df["GeoCode"].apply(lambda x : x["Latitude"] if x is not None else np.nan)
    df["LocalAuthorityName"] = df["EstablishmentDetail"].apply(lambda x : x["LocalAuthorityName"])

    df = df[["BusinessName", "Longitude", "Latitude", "LocalAuthorityName"]]

    # Checking for the TESCOS (and removing Tesco Subbrands (i.e Metro Stops) and Subtenants )
    df = df[df["BusinessName"].apply(lambda x : True if x in ["Tesco", "Tesco Superstore", "Tesco Express", "Tesco Metro", "Tesco Supermarket",'Tesco Welling Superstore'] else False)]
    df_tesco_store_locations = df_tesco_store_locations.append(df)
    

Loading Data for Municipality  501 From URL:  https://ratings.food.gov.uk/OpenDataFiles/FHRS501en-GB.xml
Loading Data for Municipality  502 From URL:  https://ratings.food.gov.uk/OpenDataFiles/FHRS502en-GB.xml
Loading Data for Municipality  503 From URL:  https://ratings.food.gov.uk/OpenDataFiles/FHRS503en-GB.xml
Loading Data for Municipality  504 From URL:  https://ratings.food.gov.uk/OpenDataFiles/FHRS504en-GB.xml
Loading Data for Municipality  505 From URL:  https://ratings.food.gov.uk/OpenDataFiles/FHRS505en-GB.xml
Loading Data for Municipality  506 From URL:  https://ratings.food.gov.uk/OpenDataFiles/FHRS506en-GB.xml
Loading Data for Municipality  507 From URL:  https://ratings.food.gov.uk/OpenDataFiles/FHRS507en-GB.xml
Loading Data for Municipality  508 From URL:  https://ratings.food.gov.uk/OpenDataFiles/FHRS508en-GB.xml
Loading Data for Municipality  509 From URL:  https://ratings.food.gov.uk/OpenDataFiles/FHRS509en-GB.xml
Loading Data for Municipality  510 From URL:  https://r

In [56]:
df_tesco_store_locations.to_csv("Geo_Data/tesco_store_locations_greater_london.csv")

In [55]:
df_tesco_store_locations

,BusinessName,Longitude,Latitude,LocalAuthorityName
1082,Tesco,0.13170400000000,51.57086600000000,Barking and Dagenham
1083,Tesco,0.07076700000000,51.53631900000000,Barking and Dagenham
1084,Tesco,0.16400500000000,51.54300800000000,Barking and Dagenham
1085,Tesco,0.15363700000000,51.55159000000000,Barking and Dagenham
1086,Tesco,0.10557000339031,51.53341293334960,Barking and Dagenham
...,...,...,...,...
3590,Tesco,-0.13382800000000,51.50658100000000,Westminster
3591,Tesco,-0.14575600000000,51.50689400000000,Westminster
3592,Tesco,-0.17762600000000,51.51892200000000,Westminster
3593,Tesco,-0.12501400000000,51.50141200000000,Westminster


In [51]:
df_tesco_store_locations["BusinessName"].unique()

array(['Tesco', 'Mitie @Tesco Metro', 'Tesco Brent Cross CR',
       'Tesco Express', 'Tesco Metro', 'Tesco Superstore',
       'Elior At Tesco Fulfillment Centre Erith',
       'Tesco Welling Superstore', 'Tesco.Com', 'Tesco Supermarket',
       'Tesco Stores Ltd', 'Costa Limited At Tesco',
       'Fresh Halal (within Tesco)', 'Greggs @Tesco',
       'NIC Services Group Ltd At Tesco.com', 'Tesco.com',
       'YO! To Go At Tesco Extra', 'Burrito Kitchen @ Tesco Metro',
       'Costa Coffee @ Tesco', 'Elior At Tesco Fulfilment Centre',
       'MITIE At Tesco Metro Ealing Broadway',
       'NIC Services Group At Tesco Vend Only Colleagues Room',
       'Servest At Tesco Metro', 'Tesco Dot Com', 'Tesco Petrol Station',
       'Elior Uk C/o Tesco Dot Com 2nd Flr', 'Tesco Extra',
       'Tesco Fresh Halal', 'Tesco Customer Cafe', 'Tesco Plc',
       'Tesco Staff Restaurant', 'Tesco Staff Canteen', 'Tesco Stores',
       'Costa Coffee @ Tesco Romford Gallows Extra',
       'Elior UK @ Tesco 